In [1]:
import panel as pn
import numpy as np

pn.extension()

# Lookup tables for air density and wind speed by height (rotor diameter)
# Both tables are derived from von Krauland et al., 2023
air_density_lookup = {
    100: 1.000,
    150: 0.995,
    200: 0.990,
    250: 0.986
}
wind_speed_lookup = {
    100: 9.54,
    150: 9.92,
    200: 10.10,
    250: 10.25
}

def annual_power_density(wind_speed, air_density=0.990, energy_pattern_factor=1.91):
    """
    Calculate the annual average power density of wind.
    """
    wind_speed = np.round(wind_speed, 2)
    power_density = 0.5 * air_density * energy_pattern_factor * (wind_speed ** 3)
    return np.rint(power_density)

def swept_area(diameter):
    """
    Calculate the swept area of a wind turbine rotor.
    """
    return np.pi * (diameter / 2) ** 2

def power_kw(power_density, rotor_diameter):
    """
    Calculate the total power output in kW given annual power density (W/m²) and rotor diameter.
    """
    area = swept_area(rotor_diameter)
    return np.rint((power_density * area) / 1000)

def annual_energy_output(power_kw_val):
    """
    Calculate the non-derated annual energy output in MWh/year from power (kW).
    """
    annual_energy_mwh = power_kw_val * 8760 / 1000
    return np.rint(annual_energy_mwh)

def derated_annual_energy_output(power_kw_val, efficiency=0.2):
    """
    Calculate the derated annual energy output in MWh/year from power (kW) and efficiency factor.
    """
    annual_energy_mwh = power_kw_val * 8760 * efficiency / 1000
    return np.rint(annual_energy_mwh)

def possible_turbine_installations(available_area_km2, rotor_diameter_m, spacing_factor=5.98):
    """
    Calculate the number of possible realizable wind turbine installations (Nturb).
    Derived from von Krauland et al., 2023 (supplemental material).
    """
    available_area_m2 = available_area_km2 * 1_000_000
    spacing_density = (spacing_factor * rotor_diameter_m) ** 2
    nturb = available_area_m2 // spacing_density
    return int(nturb)

def calculate_outputs(rotor_diameter, available_area_km2):
    air_density = air_density_lookup[rotor_diameter]
    wind_speed = wind_speed_lookup[rotor_diameter]
    pd = annual_power_density(wind_speed, air_density)
    area = swept_area(rotor_diameter)
    pk = power_kw(pd, rotor_diameter)
    energy_non_derated = annual_energy_output(pk)
    energy_derated = derated_annual_energy_output(pk)
    turbines = possible_turbine_installations(available_area_km2, rotor_diameter)
    # Site totals
    site_power = pk * turbines
    site_energy_non_derated = energy_non_derated * turbines
    site_energy_derated = energy_derated * turbines
    return {
        'air_density': air_density,
        'wind_speed': wind_speed,
        'power_density': pd,
        'swept_area': area,
        'power_kw': pk,
        'energy_non_derated': energy_non_derated,
        'energy_derated': energy_derated,
        'turbines': turbines,
        'site_power': site_power,
        'site_energy_non_derated': site_energy_non_derated,
        'site_energy_derated': site_energy_derated
    }

# Panel widgets
rotor_diameter_dropdown = pn.widgets.Select(
    name="Turbine Rotor Diameter (m)",
    options=[100, 150, 200, 250],
    value=100
)
available_area_input = pn.widgets.IntSlider(
    name="Available Area (km²)",
    start=200,
    end=4000,
    step=200,
    value=200
)

@pn.depends(rotor_diameter_dropdown, available_area_input)
def results_panel(rotor_diameter, available_area_km2):
    out = calculate_outputs(rotor_diameter, available_area_km2)
    # Top row: Air Density and Wind Speed
    top_row = pn.Row(
        pn.pane.Markdown(f"**Air Density (kg/m³):** {out['air_density']}"),
        pn.pane.Markdown(f"**Wind Speed (m/s):** {out['wind_speed']}")
    )
    # Left column: Single Turbine Output
    single_turbine_col = pn.Column(
        "### Single Turbine Output",
        f"Swept Area (m²): {out['swept_area']:,.2f}",
        f"Annual Power Density (W/m²): {out['power_density']:,}",
        f"Power Output (kW): {out['power_kw']:,}",
        f"Annual Energy Output (MWh/year, non-derated): {out['energy_non_derated']:,}",
        f"Derated Annual Energy Output (MWh/year, 20% efficiency): {out['energy_derated']:,}"
    )
    # Right column: Site Output
    site_col = pn.Column(
        "### Site Output",
        f"Installed Turbines: {out['turbines']:,}",
        f"Total Power Output (kW): {out['site_power']:,}",
        f"Total Annual Energy Output (MWh/year, non-derated): {out['site_energy_non_derated']:,}",
        f"Total Derated Annual Energy Output (MWh/year, 20% efficiency): {out['site_energy_derated']:,}"
    )
    second_row = pn.Row(single_turbine_col, site_col)
    return pn.Column(top_row, second_row)

# Tab 1: Working App
app_tab = pn.Column(
    "# Offshore Wind Power Calculator",
    "Select the turbine rotor diameter and available area below.",
    rotor_diameter_dropdown,
    available_area_input,
    results_panel
)

# Tab 2: Documentation
doc_tab = pn.Column(
    "# Documentation",
    "## App Functionality",
    "This app calculates offshore wind power outputs and possible turbine installations based on user inputs.",
    "All calculation functions are created from formulas in the book 'Harness It' by Michael Ginsberg, 2019, except for the turbine installation calculation, which is derived from von Krauland et al., 2023 (supplemental material).",
    "The lookup tables for air density and wind speed are derived from von Krauland et al., 2023.",
    "## Assumptions",
    "- Air density and wind speed are based on rotor diameter (height) using published references.",
    "- Efficiency factor for derated annual energy output is set to 0.2 (20%).",
    "- Spacing factor for turbine installations is 5.98 (offshore standard).",
    "## References",
    "- Harness It, Michael Ginsberg, 2019, Business Expert Press",
    "- von Krauland et al., 2023, United States offshore wind energy atlas. https://doi.org/10.1016/j.ecmx.2023.100410.",
    "- [Add more references here...]"
)

tabs = pn.Tabs(
    ("Calculator", app_tab),
    ("Documentation", doc_tab)
)

tabs.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Select(name='Turbine Rotor D..., options=[100, 150, 200, 250], value=100)
        [3] IntSlider(end=4000, name='Available Area (km²)', start=200, step=200, value=200)
        [4] ParamFunction(function, _pane=Column, defer_load=False)
    [1] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Markdown(str)
        [4] Markdown(str)
        [5] Markdown(str)
        [6] Markdown(str)
        [7] Markdown(str)
        [8] Markdown(str)
        [9] Markdown(str)
        [10] Markdown(str)
        [11] Markdown(str)
        [12] Markdown(str)